# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import plotly.express as px
import polars as pl
import polars.selectors as cs
from plotly.subplots import make_subplots

import patio.model.colo_core as colo_core

In [ ]:
%reload_ext autoreload

# Results

## Setup

In [ ]:
write_figs = False
run = "202507130115"

In [ ]:
results = colo_core.Results(
    "202505261158",
    "202507031241",
    "202507070053",
    "202507130115",
    max_ppa=200,
    max_attr_rev_export_clean=100,
    max_violation=0.3,
    font="Roboto",
)
results.summary_stats()

In [ ]:
results.update_params(max_violation=0.5, max_ppa=300)
results.summary_stats()

## Tables

In [ ]:
results.for_xl(clip=True)

In [ ]:
results.for_dataroom(clip=True)

## Supply curve

In [ ]:
sup_curve = results.fig_supply_curve("all", land_screen=False)
if write_figs:
    sup_curve.write_image(
        results.fig_path / f"{run}_supply_curve.pdf",
        height=580 * 0.8,
        width=1300 * 0.8,
    )
sup_curve.show()

## Opportunity map

In [ ]:
op_map = results.fig_scatter_geo("all", sixe_max=15, land_screen=False)
if write_figs:
    op_map.write_image(results.fig_path / f"{run}_best_map.pdf", width=1600, height=1200)
op_map.show()

## Selection map

In [ ]:
sel_map_fig = results.fig_selection_map(run, land_screen=False)
if write_figs:
    sel_map_fig.write_image(results.fig_path / f"{run}_sel_map.pdf", width=1600, height=1200)
sel_map_fig.show()

## Case sheets

In [ ]:
if write_figs:
    results.case_sheets(run, land_screen=False)
else:
    results.fig_case_subplots(run, land_screen=False)

## Package as csvs

In [ ]:
results.compare(
    "colo_202505261158",
    "colo_202507130115",
    values=(
        "run_status",
        "load_mw",
        "served_pct",
        "pct_load_clean",
        "ppa_ex_fossil_export_profit",
        "attr_rev_export_clean",
        "max_rolling_violation_pct",
        "good",
    ),
).filter(
    pl.col("tech").is_in(("ngcc", "nggt"))
    & pl.col("state").is_in(("NM", "CO", "AZ", "UT", "ID"))
    & (pl.col("run_status") == "SUCCESS")
).sort(
    "balancing_authority_code_eia",
    "state",
    "icx_id",
    "name",
    "regime",
).write_clipboard()

## Exploring indicator distributions

In [ ]:
px.histogram(
    results.all_summaries().filter(
        (pl.col("new_fossil_mw") > 0) & (pl.col("run_status") == "SUCCESS")
    ),
    x="max_rolling_violation_pct",
    log_y=True,
    nbins=100,
    facet_col="name",
    facet_row="run",
)

In [ ]:
px.histogram(
    results.summaries[run].filter(
        (pl.col("new_fossil_mw") > 0) & (pl.col("run_status") == "SUCCESS")
    ),
    x="max_rolling_violation_pct",
    log_y=True,
    nbins=100,
    facet_col="name",
)

## Logs